In [599]:
import numpy as np
import pandas as pd

In [600]:
total_simulation_time = 12000
mean_arrival_time = 1
mean_departure_time = 2

In [601]:

#print("\nTempo de chegada | Tipo de evento | Número de amostras\n")

n = 0
simul_time = 0
data = []
number_of_arrivals = 0

# initialize event queue
queue = []
queue.append([simul_time, np.random.exponential(scale=1/mean_arrival_time), "Chegada"])

# To calculate waiting line mean time
arrivals = []
waiting_line_time = 0

while(simul_time < total_simulation_time):
    arrival_time = np.random.exponential(scale=1/mean_arrival_time)
    departure_time = np.random.exponential(scale=1/mean_departure_time)

    event = queue.pop(0)

    if event[2] == "Chegada":
        simul_time += event[1]
        #print(f"{simul_time:8.4f} \t\t{'Chegada':14} {n:2} => {n+1:2}")
        data.append([f"{simul_time:8.4f}", f"{'Chegada'}", f"{n}", f"{n+1}"])
        queue.append([simul_time + arrival_time, arrival_time, "Chegada"])
        n += 1
        number_of_arrivals += 1

        if (n == 1):
            queue.append([simul_time + departure_time, departure_time, "Saída"])

        if (n > 1):
            arrivals.append(simul_time)
    else:
        simul_time += event[1]
        if (len(arrivals) > 0):
            waiting_line_time += simul_time - arrivals[0]
            arrivals.pop(0)
        
        #print(f"{simul_time:8.4f} \t\t{'Saída':14} {n:2} => {n-1:2}")
        data.append([f"{simul_time:8.4f}", f"{'Saída'}", f"{n}", f"{n+1}"])
        
        n -= 1
        if (n > 0):
            queue.append([simul_time + departure_time, departure_time, "Saída"])
    
    queue.sort()
    #print(queue)

print(f"\nTotal de pessoas que foram ao banco na simulação: {number_of_arrivals}")
print(f"Número de pessoas na fila ao final da simulação: {n}")
print(f"Tempo total da simulação: {simul_time:.4f}")
print(f"Tempo médio de espera na fila: {(waiting_line_time/number_of_arrivals):.4f}")



Total de pessoas que foram ao banco na simulação: 8034
Número de pessoas na fila ao final da simulação: 1
Tempo total da simulação: 12000.3202
Tempo médio de espera na fila: 1.0344
